# Fine-Tuning + Inference API 

## 1. Подготовка данных. Для обучения использован поднабор RVL-CDIP (тестовая часть), так как официальный train set недоступен. Разделение выполнено стратифицированно

In [ ]:
import os
import io

In [2]:
classes = ['invoice', 'letter', 'email', 'news_article']

In [ ]:
# os.chdir('data')
# os.mkdir('train')

In [ ]:
path = os.path.join('data', 'train')
from_path = os.path.join('data', 'test')

for c in classes:
    file = 